##### Installing all the dependencies

In [ ]:
!python -m pip install --upgrade pip
!pip install python-dotenv
!pip install --upgrade google-api-python-client
!pip install --upgrade google-auth-oauthlib google-auth-httplib2
!pip install python-decouple

##### Importing all the libraries

In [1]:
from googleapiclient.discovery import build
from decouple import config
import json
from csv import writer
import pandas as pd

##### Procedure to load keys from "credentials.json" and authenticate with Youtube API server

In [2]:
KEY = ""
with open ("credentials.json", "r") as creds:
    data = json.load(creds)
    KEY = data["KEY2"]

videoId1 = "cljQIHfdSrY" # palki sharma video 1
videoId2 = "JwHCOI8V4LQ" # palki sharma video 2
videoId3 = "OcSUWAYBPZs" # video with less comments

In [3]:
api_service_name = "youtube"
api_version ="v3"
youtube = build(api_service_name, api_version, developerKey = KEY)

In [4]:
dataFileName = "RavishYoutubeSearch2.csv"

##### Function to append a row in CSV file of the journalist.

In [5]:
def writeInCSV(record):
    with open(dataFileName, "a+", encoding="utf-8") as dataFile:
        csv_writer = writer(dataFile)
        csv_writer.writerow(record)
    return

Headers for the CSV file.

In [6]:
headingRow = [ 
    "CommentID", 
    "Timestamp", 
    "Comment", 
    "ReplyCount", 
    "LikeCount", 
#     "AuthorChannelId", 
#     "AuthorName", 
    "ParentID"
]

In [ ]:
### Calling the function to write the headers into the file.

writeInCSV(headingRow)

### Function to Fetch Comment Threads Object using "video Id" of a Youtube Video.

In [7]:
def getCommentsFromVideoID(videoId):
    commentsPerPage = 1000
    request = youtube.commentThreads().list(
        part = "snippet,replies",
        maxResults = commentsPerPage,
        videoId = videoId,
        textFormat = "plainText",
    )
    response = request.execute()
    count = 0
    
    while(response):
        count += len(response['items'])
        for commentObj in response['items']:
            commentId = commentObj['snippet']['topLevelComment']['id']
            comment = commentObj['snippet']['topLevelComment']['snippet']['textDisplay']
            replyCount = commentObj['snippet']['totalReplyCount']
            likesCount = commentObj['snippet']['topLevelComment']['snippet']['likeCount']
            timeStamp = commentObj['snippet']['topLevelComment']['snippet']['publishedAt']
            childOf = ""
            
            writeInCSV([
                commentId, 
                timeStamp, 
                comment, 
                replyCount, 
                likesCount, 
                childOf
            ])
            
            if 'replies' in commentObj:
                count += len(commentObj['replies']['comments'])
                for i in range(len(commentObj['replies']['comments'])):
                    commentId = commentObj['replies']['comments'][i]["id"].split('.')[1]
                    comment = commentObj['replies']['comments'][i]['snippet']['textDisplay']
                    childReplyCount = 0
                    likesCount = commentObj['replies']['comments'][i]['snippet']['likeCount']
                    timeStamp = commentObj['replies']['comments'][i]['snippet']['publishedAt']
                    childOf = commentObj['replies']['comments'][i]['snippet']['parentId']

                    writeInCSV([
                        commentId, 
                        timeStamp, 
                        comment, 
                        childReplyCount, 
                        likesCount, 
                        childOf
                    ])
    
        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part = 'snippet,replies',
                videoId = videoId,
                maxResults= commentsPerPage,
                pageToken = response['nextPageToken']
            ).execute()
        else:
            break
    return count

In [7]:
def getCommentsFromVideoID(videoId):
    commentsPerPage = 1000
    request = youtube.commentThreads().list(
        part = "snippet,replies",
        maxResults = commentsPerPage,
        videoId = videoId,
        textFormat = "plainText",
    )
    response = request.execute()
    count = 0
    
    while(response):
        count += len(response['items'])
        for commentObj in response['items']:
            commentId = commentObj['snippet']['topLevelComment']['id']
            comment = commentObj['snippet']['topLevelComment']['snippet']['textDisplay']
            replyCount = commentObj['snippet']['totalReplyCount']
            likesCount = commentObj['snippet']['topLevelComment']['snippet']['likeCount']
            timeStamp = commentObj['snippet']['topLevelComment']['snippet']['publishedAt']
            childOf = ""
            
            writeInCSV([commentId, timeStamp, comment, replyCount, likesCount, childOf])
            
            if 'replies' in commentObj:
                count += len(commentObj['replies']['comments'])
                for i in range(len(commentObj['replies']['comments'])):
                    commentId = commentObj['replies']['comments'][i]["id"].split('.')[1]
                    comment = commentObj['replies']['comments'][i]['snippet']['textDisplay']
                    childReplyCount = 0
                    likesCount = commentObj['replies']['comments'][i]['snippet']['likeCount']
                    timeStamp = commentObj['replies']['comments'][i]['snippet']['publishedAt']
                    childOf = commentObj['replies']['comments'][i]['snippet']['parentId']

                    writeInCSV([commentId, timeStamp, comment, childReplyCount, likesCount, childOf])
    
        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part = 'snippet,replies',
                videoId = videoId,
                maxResults= commentsPerPage,
                pageToken = response['nextPageToken']
            ).execute()
        else:
            break
    return count

##### 'queries' holds the list of queries to be searched on Youtube.

In [8]:
queries = [
    "palki sharma upadhyay interview", 
    "ravish kumar biography journey", 
    "arnab goswami interview life journey", 
    "anjana om kashyap journey biography"
]


Limit on the maximum number of comments to be extracted via API and procedures above.

In [9]:
maxCommentCount = 36000

### Building the dataset manually
Each of the below cells has to be run again and again for each of the queries declared above as the connection between the system and Youtube is breaks.

In [10]:
### Making the search call to the API to bring the videos that appear on SERP.

request = youtube.search().list(
    part="snippet",
    maxResults = 100,
    q="ravish kumar biography journey",
    order="relevance",
#     regionCode="IND"
)
response = request.execute()
# print(response)

In [11]:
### Storing the Search Call object inside a json file.

with open ("responseRavish.json", "w") as fout:
    jsonObj = json.dump(response, fout)

In [12]:
### Reading the json file and extracting out the 'video id's from the object.

with open("responseRavish.json", "r") as fread:
    response = json.load(fread)

videosIDS = list()
for i in range(len(response["items"])):
    videosIDS.append(response["items"][i]["id"]["videoId"])

### open a file for jouralist.

dataFileName = "".join(queries[1].split()[:2])+".csv"
writeInCSV(headingRow)

In [13]:
### Declaring the variables as 0 that will control the loop later.

vid = 0
commentCount = 0

In [14]:
### The while loop that fetches the comments from the videos.

while(vid < len(videosIDS) and commentCount < maxCommentCount):
    commentCount += getCommentsFromVideoID(videosIDS[vid])
    vid += 1

In [15]:
### checking the values of variable.

print(maxCommentCount, commentCount)

36000 45250


### Attempt to build Function that automates fetching comments
Here is the attempt to build the function that makes the youtube search call as well as the comment thread call to build the dataset sequentially and automatically without much interference.

But there is a duration window associated the API for a client that could not be handled by this procedure. Hence it was not a success.

In [ ]:

for query in queries:
    commentCount = 0
    
    # Making the request to get video id list.
    request = youtube.search().list(
        part="snippet",
        maxResults = 100,
        q=query,
    )
    response = request.execute()
    print("response for query: ", query, " has been fetched!")
    
    # Making list of video ids for iteration.
    videosIDS = list()
    for i in range(len(response["items"])):
        videosIDS.append(response["items"][i]["id"]["videoId"])
    
    # open a file for jouralist.
    dataFileName = "".join(query.split()[:2])+".csv"
    writeInCSV(headingRow)
    
    vid = 0
    while(vid < len(videosIDS) and commentCount < maxCommentCount):
        commentCount += getCommentsFromVideoID(videosIDS[vid])
        vid += 1
    print("Comment Extraction for the query completed.")

### Extending the purpose to trying the data cleaning.

In [ ]:
!pip install emoji

In [ ]:
import emoji

In [ ]:
print(emoji.emojize('Python is :thumbs_up:'))